#Perform a Sequential chain in Langchain

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import SequentialChain, LLMChain
from langchain.schema.runnable import RunnablePassthrough

In [2]:
# using the Chat OpenAI GPT 4o-mini
llm = ChatOpenAI(temperature=0.0, model="gpt-4o-mini")

In [6]:
# List of Prompts to be used
original_review_prompt = """ 
Translate the below review in English. If it is already in English, skip the translation and output the same review.
Review: {review}
"""
prompt1 = ChatPromptTemplate.from_template(original_review_prompt)

In [7]:
summary_creation_prompt = """ 
Create a concise summary of the following text, ensuring that the summary captures the main points and essential information.
Text: {english_review}
"""
prompt2 = ChatPromptTemplate.from_template(summary_creation_prompt)

In [8]:
language_check_prompt =""" 
Identify the language of the following review:
\n\n
{review}
"""
prompt3 = ChatPromptTemplate.from_template(language_check_prompt)

In [9]:
find_sentiment_prompt = """
You are a sentiment analysis expert. You will be given a sentence and you need to find the sentiment of the sentence.
The sentiment can be one of the following Review:
1. Positive 2. Negative 3. Neutral
\n\n
Review: {review}
English Review: {english_review}
Summary: {summary}
Original Review Language: {review_language}

Determine the sentiment of the review and return the result in the following format:
Sentiment: <sentiment>
\n\n
"""

prompt4 = ChatPromptTemplate.from_template(find_sentiment_prompt)

In [10]:

build_email_response_prompt = """
You are an expert email design assistant and good customer service agent. Your task is to generate a professional and engaging consise email response based on the customer review.
Thank them for their feedback in the response. If in case of any negative feedback, make it more polite, engaging and professional. 
Sign the email with 'Sincerely, [Your Name]' at the end.
Use the below details.

Review: {review}
English Review: {english_review}
Summary: {summary}
Original Review Language: {review_language}
Sentiment: {sentiment}
"""
prompt5 = ChatPromptTemplate.from_template(build_email_response_prompt)

In [11]:
# Now goes the chains
output_parser = StrOutputParser()
original_review_chain = prompt1 | llm | output_parser
summary_creation_chain = prompt2 | llm | output_parser
language_check_chain = prompt3 | llm | output_parser
sentiment_check_chain = prompt4 | llm | output_parser
build_email_chain = prompt5 | llm | output_parser

In [12]:
final_chain = ({"review":RunnablePassthrough(),"english_review":original_review_chain}
               |RunnablePassthrough.assign(summary = summary_creation_chain)
               |RunnablePassthrough.assign(review_language = language_check_chain)
               |RunnablePassthrough.assign(sentiment = sentiment_check_chain)
               |RunnablePassthrough.assign(automated_email =build_email_chain)
               )

In [13]:
review = """ Titre: Déçu par la qualité et le confort

Avis: Je suis déçu par cette chemise. La coupe est trop serrée, bien que j'aie commandé ma taille habituelle. Le tissu est rigide et inconfortable, surtout après le lavage. La qualité ne vaut pas le prix. Je ne recommande pas cet achat."""

In [14]:
result = final_chain.invoke(review)
result

{'review': " Titre: Déçu par la qualité et le confort\n\nAvis: Je suis déçu par cette chemise. La coupe est trop serrée, bien que j'aie commandé ma taille habituelle. Le tissu est rigide et inconfortable, surtout après le lavage. La qualité ne vaut pas le prix. Je ne recommande pas cet achat.",
 'english_review': 'Title: Disappointed by the quality and comfort\n\nReview: I am disappointed with this shirt. The fit is too tight, even though I ordered my usual size. The fabric is stiff and uncomfortable, especially after washing. The quality is not worth the price. I do not recommend this purchase.',
 'summary': 'The reviewer expresses disappointment with the shirt, citing a tight fit despite ordering the usual size, stiff and uncomfortable fabric after washing, and overall poor quality for the price. They do not recommend the purchase.',
 'review_language': 'The language of the review is French.',
 'sentiment': 'Sentiment: Negative',
 'automated_email': "Subject: Thank You for Your Feedb